In [40]:
import torch

def initialize(n, seed, device):
    torch.manual_seed(seed)
    spins = torch.tensor([1 if torch.rand(1).item() < 0.5 else -1 for _ in range(n)], device=device)
    J = torch.normal(mean=0.0, std=1.0, size=(n, n), device=device)
    J = torch.tril(J, diagonal=-1) + torch.tril(J, diagonal=-1).T  # 下三角のみ
    h = torch.zeros(n, device=device)  # 外部磁場
    st = torch.ones(n, device=device) * calculate_energy(spins, h, J)
    st1 = st * ((spins + 1) / 2)
    return spins, h, J, st, st1

def decision_probabilities(st, st1, alpha):
    return alpha * ((st1 / st) - 0.5) + 0.5

def calculate_energy(spins, h, J):
    return -(torch.sum(h * spins) + (torch.sum(J * torch.outer(spins, spins)) / (len(spins) - 1)))



In [53]:
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
start_seed = 42
seed = 42
iter = 1e6
alpha = torch.tensor(0.0, device=device)
alpha_inc = torch.tensor(1/iter, device=device)
energy_series = []

spins, h, J, st, st1 = initialize(n, seed, device)

for i in range(int(iter)):
    torch.manual_seed(seed)
    spins = 2 * (decision_probabilities(st, st1, alpha) > torch.rand(n, device=device)) - 1
    st += torch.exp(calculate_energy(spins, h, J))
    st1 += st * ((spins + 1) / 2)
    alpha += alpha_inc
    energy_series.append(calculate_energy(spins, h, J).cpu().item())
    seed += 1

energy_series.insert(0, 'energy')
file_name = 'hoge'
output.list_to_csv(file_name, energy_series, start_seed, iter)

In [37]:
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
start_seed = 42
seed = 42
iter = 1e6
alpha = torch.tensor(0.0, device=device)
alpha_inc = torch.tensor(1/iter, device=device)
energy_series = []
m_series = []

spins, h, J, st, st1 = simulation.initialize(n, seed, device)

for i in range(int(iter)):
    torch.manual_seed(seed)
    spins = 2 * (simulation.decision_probabilities(st, st1, alpha) > torch.rand(n, device=device)) - 1
    st += torch.exp(simulation.calculate_energy(spins, h, J))
    st1 += torch.exp(simulation.calculate_energy(spins, h, J)) * ((spins + 1) / 2)
    z = (st1 / st)
    alpha += alpha_inc
    m = 2 * alpha * (z - 0.5)
    energy_series.append(simulation.calculate_energy(spins, h, J).cpu().item())
    m_series.append(m.tolist())
    seed += 1

energy_series.insert(0, 'energy')
energy_file_name = 'tauinf_energy'
m_file_name = 'tauinf_m'
output.list_to_csv(energy_file_name, energy_series, start_seed, iter)
output.matrix_to_csv(m_file_name, m_series, start_seed, iter)

In [29]:
import simulation
import torch
import importlib
importlib.reload(simulation)

n = 3
seed = 42
alpha =  0.5
device = "cuda"
spins, h, J, st, st1 = simulation.initialize(n, seed, device)

print(st)
print(st1)

spins = 2 * (simulation.decision_probabilities(st, st1, alpha) > torch.rand(n, device=device)) - 1
st += torch.exp(simulation.calculate_energy(spins, h, J))
st1 += torch.exp(simulation.calculate_energy(spins, h, J)) * ((spins + 1) / 2)
print(st)
print(st1)


tensor([0.0987, 0.0987, 0.0987], device='cuda:0')
tensor([0.0000, 0.0000, 0.0987], device='cuda:0')
tensor([2.8640, 2.8640, 2.8640], device='cuda:0')
tensor([0.0000, 2.7653, 2.8640], device='cuda:0')


In [27]:
a = torch.ones(2) * 0.5
b = torch.tensor([1, 2])
c = torch.rand(2)
2 * (a > c) - 1

tensor([1, 1])

In [14]:
spins = torch.tensor([1 if torch.rand(1).item() < 0.5 else -1 for _ in range(2)])
print(spins)
print((spins + 1)/2)
print(b * ((spins + 1)/2))

tensor([-1,  1])
tensor([0., 1.])
tensor([0., 2.])


In [36]:
a = torch.tensor([2, 1])
b = torch.tensor([1, 2])
c = []
c.append(a.tolist())
print(c)

[[2, 1]]
